## gensim으로 뉴스 기사 토픽 모델링

### 뉴스 기사 수집

In [1]:
import requests
from bs4 import BeautifulSoup
import pickle

In [2]:
def get_daum_news_content(news_id):
    url = 'https://news.v.daum.net/v/{}'.format(news_id)
    resp = requests.get(url)
    soup = BeautifulSoup(resp.text)
    
    content = ''
    for p in soup.select('div#harmonyContainer p'):
        content += p.get_text()
    return content


news_contents = []
news_ids = ['20190728165812603', '20201125113816419', '20201125093706258', '20201121070335100', '20201118084434035']

page_contents = []
for id in news_ids:
    text = get_daum_news_content(id)
    page_contents.append(text)


news_contents.append(page_contents)

print(news_contents[0][1])
print(len(page_contents))

미국 전기차업체 테슬라 주가가 질주하며 24일(현지시간) 시가총액 5000억달러(약 555조원)를 돌파했다. 테슬라 주가 급등으로 지난주 세계 3위 부자에 올랐던 일론 머스크 최고경영자(CEO)는 1주일 만에 순위를 2위로 끌어올렸다.이날 뉴욕증시에서 테슬라는 전거래일 대비 6.43% 오른 주당 555.38달러에 거래를 마치며 사상 최고가를 기록했다. 주가 527.48달러가 시총 5000억달러의 분기점이었는데, 이를 훌쩍 넘었다. 테슬라 주가는 말그대로 질주하고 있다. 코로나19 팬데믹 이전인 지난 1월22일 시총 1000억달러를 처음 넘은 이후 10개월여 만에 5배 이상 불어났다. 일부 우려에도 불구, 실적을 숫자로 보여주며 승승장구하고 있는 것. 경제전문매체 CNBC에 따르면 테슬라는 올 3분기에 창립 후 최대 규모인 13만9300대의 차량을 인도했다. 판매량 증가는 실적 호조로 이어져 올 3분기까지 5개 분기 연속 흑자를 냈다.여러 변수도 테슬라 주가의 추가 상승을 예고하고 있다. 우선, 다음달 21일 스탠더드앤드푸어스(S&P)500 지수 편입이 예정돼 있다. 이에 따른 패시브 자금 유입이 기대된다. 모건스탠리는 지난주 테슬라에 대한 투자 의견을 ‘비중 확대’로 상향 조정했다.조 바이든 미 대통령 당선인의 환경 중시 정책으로 전기차 랠리가 이어지고 있는 점도 우호적 요인이다. 전기차 대장주 테슬라가 최대 수혜주로 부각되고 있다.테슬라 목표주가를 1000달러까지 제시한 증권사도 나왔다. 미국 웨드부시증권은 최근 향후 수년간 전기차 수요가 늘 것이라며 테슬라의 목표주가를 800달러에서 1000달러로 올렸다.테슬라 주가가 급등하면서 머스크 CEO는 제프 베이조스 아마존 CEO에 이어 세계 2위 부자에 올랐다. 지난주 마크 저커버그 페이스북 CEO를 제치고 3위에 오른데 이어, 빌 게이츠 MS 창업자까지 제쳤다.
5


In [3]:
with open("data/daum_news_content.pk", "wb") as f:
    pickle.dump(news_contents, f)

### 1. 토픽 모델링을 위한 라이브러리 불러오기

In [4]:
from tqdm import tqdm_notebook
import numpy as np
import string
import re
import warnings

import MeCab   # 형태소 분석기
mecab = MeCab.Tagger()

from gensim import corpora
from gensim import models

import matplotlib.pyplot as plt
%matplotlib inline
warnings.filterwarnings("ignore", category=DeprecationWarning)

### 2. 텍스트 전처리 함수 만들기

In [5]:
def mecab_nouns(text):
    nouns = []

    # 원하는 TOKEN\tPOS의 형태를 추출하는 정규표현식.
    pattern = re.compile(".*\t[A-Z]+")

    # 패턴에 맞는 문자열을 추출하여 konlpy의 mecab 결과와 같아지도록 수정.
    temp = [tuple(pattern.match(token).group(0).split("\t")) for token in mecab.parse(text).splitlines()[:-1]]

    # 추출한 token중에 POS가 명사 분류에 속하는 토큰만 선택.
    for token in temp:
        if token[1] == "NNG" or token[1] == "NNP" or token[1] == "NNB" or token[1] == "NNBC" or token[1] == "NP" or \
                token[1] == "NR":
            nouns.append(token[0])

    return nouns

def mecab_morphs(text):
    morphs = []

    # 원하는 TOKEN\tPOS의 형태를 추출하는 정규표현식.
    pattern = re.compile(".*\t[A-Z]+")

    # 패턴에 맞는 문자열을 추출하여 konlpy의 mecab 결과와 같아지도록 수정.
    temp = [tuple(pattern.match(token).group(0).split("\t")) for token in mecab.parse(text).splitlines()[:-1]]

    # 추출한 token중에 문자열만 선택.
    for token in temp:
        morphs.append(token[0])

    return morphs

In [6]:
def read_documents(input_file_name):
    corpus = []
    
    with open(input_file_name, 'rb') as f:
        temp_corpus = pickle.load(f)
        
    for page in temp_corpus:
        corpus += page
    
    return corpus

def text_cleaning(docs):
    # 한국어를 제외한 글자를 제거하는 함수.
    for doc in docs:
        doc = re.sub("[^ㄱ-ㅎㅏ-ㅣ가-힣 ]", "", doc)

    return docs

def define_stopwords(path):    
    SW = set()
    
    with open(path, encoding='utf8') as f:
        for word in f:
            SW.add(word)

    return SW

def text_tokenizing(corpus, tokenizer):
    token_corpus = []

    if tokenizer == "noun":
        for n in tqdm_notebook(range(len(corpus)), desc="Preprocessing"):
            token_text = mecab_nouns(corpus[n])
            token_text = [word for word in token_text if word not in SW and len(word) > 1]
            token_corpus.append(token_text)
            
    elif tokenizer == "morph":
        for n in tqdm_notebook(range(len(corpus)), desc="Preprocessing"):
            token_text = mecab_morphs(corpus[n])
            token_text = [word for word in token_text if word not in SW and len(word) > 1]
            token_corpus.append(token_text)

    elif tokenizer == "word":
        for n in tqdm_notebook(range(len(corpus)), desc="Preprocessing"):
            token_text = corpus[n].split()
            token_text = [word for word in token_text if word not in SW and len(word) > 1]
            token_corpus.append(token_text)
        
    return token_corpus


input_file_name = "data/daum_news_content.pk"
documents = read_documents(input_file_name)

SW = define_stopwords("stopwords-ko.txt")
cleaned_text = text_cleaning(documents)

tokenized_text = text_tokenizing(cleaned_text, tokenizer="noun") # tokenizer = "noun" or "word"

print(tokenized_text[0])


['아시아', '경제', '이민우', '기자', '머스크', '테슬라', '최고경영자', '자사', '전기', '자동차', '모델', '넷플릭스', '유튜브', '온라인', '동영상', '서비스', '탑재', '예고', '자율', '주행', '전기', '주행', '정보', '각종', '영상', '콘텐츠', '공간', '확장', '전략', '풀이', '현지', '시간', '버지', '주요', '외신', '머스크', '자신', '트위터', '계획', '자동차', '정차', '넷플릭스', '유튜브', '감상', '기능', '추가', '편안', '좌석', '서라운드', '사운드', '오디오', '영화관', '느낌', '강조', '테슬라', '콘텐츠', '방면', '확장', '이번', '처음', '지난달', '세계', '최대', '게임', '운전자', '폴아웃', '게임', '발표', '이후', '최근', '게임', '업체', '아타', '리사', '자동차', '경주', '게임', '포지션', '슈팅', '게임', '템페스트', '미사일', '커맨드', '고전', '게임', '제공', '운전대', '게임', '조작', '방식', '주차', '경우', '이번', '영상', '콘텐츠', '주행', '감상', '방안', '고려', '테슬라', '규제', '당국', '자율', '주행', '승인', '차량', '승객', '동영상', '설명', '자율', '주행', '안전', '우려', '상황', '차량', '공유', '서비스', '우버', '자율', '주행', '시범', '차량', '보행자', '충돌', '사고', '발생', '당시', '시험', '운전자', '디즈니', '동영상', '스트리밍', '서비스', '이용', '이민우', '기자']


문서 읽기의 과정은 앞서 단어 임베딩의 경우와 다르지 않다. 다음 과정은 문서-단어 행렬을 만드는 과정이다.

### 3. 토픽 모델링에 사용할 함수들 확인하기

In [7]:
len(tokenized_text)

5

In [8]:
# 문서-단어 행렬 만들기
# 어휘(vocabulary) 학습
dictionary = corpora.Dictionary(tokenized_text)

# 문서-단어 행렬(document-term matrix) 생성 = sklarn.count-vectorizer와 동일
corpus = [dictionary.doc2bow(text) for text in tokenized_text]

In [9]:
print(dictionary)

Dictionary(378 unique tokens: ['각종', '감상', '강조', '게임', '경우']...)


In [10]:
corpus[0][:5]   # (index, word-frequency)

[(0, 1), (1, 2), (2, 1), (3, 7), (4, 1)]

In [11]:
# TFIDF 문서-단어 행렬 생성
tfidf = models.TfidfModel(corpus)
corpus_tfidf = tfidf[corpus]
corpus_tfidf[0][:5]

[(0, 0.07480649367196546),
 (1, 0.14961298734393091),
 (2, 0.07480649367196546),
 (3, 0.5236454557037582),
 (4, 0.023743117703705182)]

In [12]:
# LDA model
model = models.ldamodel.LdaModel(corpus, num_topics=4, id2word=dictionary)

In [13]:
model.show_topic(topicid=0, topn=10)

[('테슬라', 0.023975313),
 ('보험', 0.020588325),
 ('보험료', 0.020287853),
 ('차량', 0.018366486),
 ('제시', 0.017672697),
 ('주행', 0.01587173),
 ('자율', 0.014155747),
 ('전기차', 0.014036418),
 ('소비자', 0.01108369),
 ('자동차', 0.010765007)]

In [14]:
model.show_topic(topicid=2, topn=10)

[('테슬라', 0.043347653),
 ('에어컨', 0.019266868),
 ('주가', 0.018104866),
 ('달러', 0.015998453),
 ('머스크', 0.013409254),
 ('세계', 0.01090842),
 ('가정', 0.010259191),
 ('전기차', 0.008831249),
 ('에너지', 0.008336654),
 ('저커버그', 0.007808477)]

### 4. 토픽 모델링을 추가하여 코드 완성하기

In [15]:
NUM_TOPICS = 3
NUM_TOPIC_WORDS = 30

def build_doc_term_mat(documents):
    # 문서-단어 행렬 만들어주는 함수.
    dictionary = corpora.Dictionary(documents)
    corpus = [dictionary.doc2bow(document) for document in documents]
        
    return corpus, dictionary


def print_topic_words(model):
    # 토픽 모델링 결과를 출력해 주는 함수.
    for topic_id in range(model.num_topics):
        topic_word_probs = model.show_topic(topic_id, NUM_TOPIC_WORDS)
        
        print("Topic ID: {}".format(topic_id))
        for topic_word, prob in topic_word_probs:
            print("\t{}\t{}".format(topic_word, prob))
            
        print("\n")

        
# document-term matrix를 만들고,
corpus, dictionary = build_doc_term_mat(tokenized_text)

# LDA를 실행.
model = models.ldamodel.LdaModel(corpus, num_topics=NUM_TOPICS, id2word=dictionary, alpha="auto", eta="auto")

# 결과를 출력.
print_topic_words(model)

Topic ID: 0
	테슬라	0.029563775286078453
	에어컨	0.02186998352408409
	가정	0.013844176195561886
	머스크	0.01099263597279787
	달러	0.010756781324744225
	닛케이	0.009071593172848225
	에너지	0.008136250078678131
	주가	0.00780560914427042
	전기	0.007796871475875378
	세계	0.0076948800124228
	가능	0.007597193121910095
	저커버그	0.007148283533751965
	부자	0.00658964179456234
	제휴	0.006577902007848024
	생산	0.006550389342010021
	차량	0.006527837831526995
	절약	0.006504897028207779
	재산	0.006036144215613604
	지수	0.005630275700241327
	설명	0.005564847029745579
	게임	0.005531303118914366
	효율	0.005472053773701191
	자동차	0.005287159699946642
	자율	0.005266539286822081
	만큼	0.005227029789239168
	일반	0.005191377829760313
	전기차	0.004932462703436613
	주행	0.004928546492010355
	시작	0.004891617223620415
	배터리	0.004850378260016441


Topic ID: 1
	테슬라	0.031250324100255966
	주가	0.021136265248060226
	달러	0.01895815320312977
	게임	0.013170955702662468
	주행	0.01134065818041563
	머스크	0.009518957696855068
	전기차	0.008369664661586285
	분기	0.008300184272229671
	차량	0.0082016559317

### 5. pyLDAvis를 통한 토픽 모델링 결과 시각화하기

In [ ]:
import pyLDAvis
import pyLDAvis.gensim

# pyLDAvis를 jupyter notebook에서 실행할 수 있게 활성화.
pyLDAvis.enable_notebook()

data = pyLDAvis.gensim.prepare(model, corpus, dictionary)
data

![LDA](./data/text_LDA.jpg)

https://github.com/woosa7/nbcc_on_campus/blob/main/ML/KoNLPy/data/text_LDA.jpg